In [24]:
import sys

import pylibxc
import numpy as np
import matplotlib.pyplot as plt

def get_density(r_s):
  return 3/(4*np.pi*(r_s ** 3))

def hartree_to_mRy(energy):
  return energy * 2 * 1000

def get_eps_x_unif(n):
  return -(3/(4*np.pi))*((n*3*np.pi**2)**(1/3))

def get_grad_n(s, n):
  return s*((n**(4/3))*2*(3*np.pi**2)**(1/3))

def get_up_dn_density(n, zeta):
  n = np.expand_dims(n.flatten(), axis=1)
  zeta = np.expand_dims(zeta.flatten(), axis=1)

  up_density = ((zeta * n) + n)/2
  dn_density = (n - (zeta * n))/2
  return np.concatenate((up_density, dn_density), axis=1)

def get_tau(alpha, grad_n, n):
  tau_w = (grad_n**2) / (8*n)
  tau_unif = (3/10)*((3*np.pi)**(2/3))*(n**(5/3))
  
  return alpha*tau_unif +tau_w

eps_x = eps_c = 'zk'


In [25]:
# polarized PBE

# build functional
pbe_x = pylibxc.LibXCFunctional("gga_x_pbe", "polarized")
pbe_c = pylibxc.LibXCFunctional("gga_c_pbe", "polarized")

r_s = np.array([0.5, 1, 2, 3, 4, 5, 6, 10, 20, 40, 60, 100])
#r_s = np.array([0.01, 2, 10, 1000])
r_s = np.linspace(0.1, 100, num=500)


s = np.linspace(0, 5, num=50)

zeta = np.array([1])
zeta = np.linspace(0, 1, num=50)

m_r_s, m_s, m_zeta = np.meshgrid(r_s, s, zeta, indexing='ij')

m_n = get_density(m_r_s)
m_sigma = get_grad_n(m_s, m_n)
m_eps_x_unif = get_eps_x_unif(m_n)
m_n_spin = get_up_dn_density(m_n, m_zeta)

# create input
inp = {}

# density
inp["rho"] = m_n_spin

# (| \nabla n |^2, 0, 0) to follow libxc convention
sigma = np.expand_dims(m_sigma.flatten()**2, axis=1)
sigma = np.concatenate((sigma, np.zeros_like(sigma), np.zeros_like(sigma)), axis=1)
inp["sigma"] = sigma


In [26]:
# results in a.u.
pbe_x_res = pbe_x.compute(inp)
pbe_c_res = pbe_c.compute(inp)

In [27]:
# processing 

pbe_x_res[eps_x] = np.squeeze(pbe_x_res[eps_x])
pbe_c_res[eps_c] = np.squeeze(pbe_c_res[eps_c])

pbe_xc_res = pbe_c_res[eps_c] #+ pbe_x_res[eps_x]
pbe_xc_res = pbe_xc_res.reshape(m_r_s.shape)

# F_xc
pbe_xc_res /= m_eps_x_unif

In [28]:
# check all results
# F_c(r_s', ...) > F_c(r_s, ...) for r_s' > r_s

min_diff = np.amin(np.diff(pbe_xc_res, axis=0))
print('min_diff = ', min_diff)

# if min_diff > 0, then condition is satisified

min_diff =  4.76304702078361e-06
